In [2]:
import requests

# Variabel yang diperlukan
protocol = "https"
ip_or_domain = "platform.antares.id"
port = "8443"
cse_id = "antares-id"
cse_name = "antares-cse"
application_name = "AMIModule"
device_name = "LoRa_Gateway"
access_key = "fbcc54e8fa44769a:601102b2dae28971"
initial_time = "20250508T000000"
last_time = "20250510T000000"

# Membangun URL
url = f"{protocol}://{ip_or_domain}:{port}/~/{cse_name}/{cse_id}/{application_name}/{device_name}?lim=400&fu=1&drt=2&ty=4"
# Header yang diperlukan
headers = {
    "Accept": "application/json",
    "X-M2M-Origin": access_key,
    "Content-Type": "application/json"
}

# Mengirim GET request
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Memeriksa apakah ada error dalam respons

    # Mengambil data JSON dari respon
    data = response.json()
    print("Raw JSON yang diterima")
    # print(data)

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except requests.exceptions.RequestException as err:
    print(f"Error occurred: {err}")


Raw JSON yang diterima


In [4]:
import pandas as pd
import json
from datetime import datetime

def convert_to_dataframe(json_data):
    if not json_data or "m2m:list" not in json_data:
        print("Invalid JSON structure")
        return pd.DataFrame()
    
    records = []
    for item in json_data["m2m:list"]:
        cin = item.get("m2m:cin", {})
        con_raw = cin.get("con", "{}")

        # Bersihkan karakter kontrol yang mungkin menyebabkan error
        con_raw = con_raw.replace("\n", "").replace("\t", "").replace("\r", "")

        try:
            con_data = json.loads(con_raw, strict=False)  # Gunakan strict=False untuk mengabaikan error karakter kontrol
            ct_raw = cin.get("ct", "")  # Tambah kolom received_at_server
            formatted_ct = datetime.strptime(ct_raw, "%Y%m%dT%H%M%S").strftime("%Y-%m-%d %H:%M:%S")
            con_data["received_at_server"] = formatted_ct # Tambah kolom received_at_server
            
            records.append(con_data)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
            print(f"Problematic data: {con_raw}")

    return pd.DataFrame(records)

df = convert_to_dataframe(data)
df.head()

,devUI,time_at_device,time_received_at_gateway,time_send_from_gateway,RSSI,protocol,voltage,current,power,energy,frequency,power_factor,received_at_server
0,08:D1:F9:35:B5:48,2025-05-11 06:29:24,2025-05-11 06:29:40,2025-05-11 06:29:40.542,-54,mqtt,nan,nan,nan,nan,915000000.0,nan,2025-05-11 06:29:40
1,08:D1:F9:35:B5:48,2025-05-11 06:29:13,2025-05-11 06:29:29,2025-05-11 06:29:29.001,-73,http,nan,nan,nan,nan,915000000.0,nan,2025-05-11 06:29:31
2,08:D1:F9:35:B5:48,2025-05-11 06:29:03,2025-05-11 06:29:19,2025-05-11 06:29:19.267,-81,http,nan,nan,nan,nan,915000000.0,nan,2025-05-11 06:29:20
3,08:D1:F9:35:B5:48,2025-05-11 06:28:52,2025-05-11 06:29:08,2025-05-11 06:29:08.547,-88,http,nan,nan,nan,nan,915000000.0,nan,2025-05-11 06:29:09
4,08:D1:F9:35:B5:48,"2025-05-11 06:28:41 ,",2025-05-11 06:28:57,2025-05-11 06:28:57.853,-99,mqtt,,,,,nan,nan,2025-05-11 06:28:57
